# 🛰️ BIOMASS L2 Product Analysis and Visualization 🌳

(c) European Space Agency (ESA) Licensed under ESA Software Community Licence Permissive (Type 3) – v2.4

# Execution and scope
The following libraries are needed to execute this notebook: rasterio (gdal), netCDF4, numpy, scipy, matplotlib, xml. Environment files tested on win/linux machines can be provided upon request (just send an email to muriel.pinheiro@esa.int) 

#### The aim of the notebook
This notebook is intended to demonstrate the processing and visualization of BIOMASS L2 products, including:
- Reading and visualizing COG-formatted SAR measurement data (data and quality)
- Extracting and displaying LUT (Look-Up Table) variables from the annotated NetCDF datasets for L2A

#### Replication of notebook
This notebook can be replicated/executed starting from the shared TDS. For further details, you can consult the shared documentation or simply send an email to muriel.pinheiro@esa.int

## 🗂️ Index

1. [🌿 Section 1: L2A FH](#🌿-section-1-l2a-fh)
2. [🌳 Section 2: L2A FD](#🌳-section-2-l2a-fd)
3. [🍃 Section 3: L2A GN](#🍃-section-3-l2a-gn)
4. [🌲 Section 4: L2B FH](#🌲-section-4-l2b-fh)
5. [🌴 Section 5: L2B FD](#🌴-section-5-l2b-fd)
6. [🎋 Section 6: L2B AGB](#🎋-section-6-l2b-agb)


In [ ]:
%matplotlib inline

In [ ]:
import os
import rasterio
import netCDF4
import numpy as np
import matplotlib.pyplot as plt
import warnings
import BiomassProduct 
from pathlib import Path
warnings.filterwarnings('ignore', category=DeprecationWarning)
print()
folder_TDS =Path("../L2A/")
print(os.listdir(folder_TDS) )
print(folder_TDS)
print("CWD:", Path.cwd())

# Cerca le tre cartelle L2A
print("FH dirs:", list(folder_TDS.glob("BIO_FP_FH__L2A_*")))
print("FD dirs:", list(folder_TDS.glob("BIO_FP_FD__L2A_*")))
print("GN dirs:", list(folder_TDS.glob("BIO_FP_GN__L2A_*")))


paths = BiomassProduct.L2AProductPaths(folder_TDS)  

path_FH=Path("../L2A/BIO_FP_TFH_L2A_20170125T163833_20170206T163906_I_G03_M03_C___T000_F001_02_D2YZZR")
product_fd_l2a = BiomassProduct.l2a_fd(paths.fd)
product_gn_l2a = BiomassProduct.l2a_gn(paths.gn)    
product_fh_l2a = BiomassProduct.l2a_fh(paths.fh)


## 🌿 Section 1: L2A FH

#### Section 1.1: L2A FH COG 

Get the COGs in the measurement folder

In [ ]:
l2a_fh = product_fh_l2a.measurement_file
l2a_fh_quality = product_fh_l2a.measurement_quality_file

plot FH and its quality layer

In [ ]:
# get the rasters
nan_value=product_fh_l2a.floatNoDataValue

with rasterio.open(l2a_fh) as fh_raster:
    fh = fh_raster.read(1)
    
    # Get the coordinates for the plot
    transform = fh_raster.transform
    width = fh_raster.width
    height = fh_raster.height
    cols, rows = np.meshgrid(np.arange(width), np.arange(height))
    xs, ys = rasterio.transform.xy(transform, rows, cols)
    
    # Convert to numpy arrays
    lons = np.array(xs)
    lats = np.array(ys)



with rasterio.open(l2a_fh_quality) as fh_quality_raster:
    fh_quality = fh_quality_raster.read(1)

fh_avg = np.mean(np.ma.masked_equal(fh,nan_value))
fh_std = np.std(np.ma.masked_equal(fh, nan_value))

fh_quality_avg = np.mean(np.ma.masked_equal(fh_quality, nan_value))
fh_quality_std = np.std(np.ma.masked_equal(fh_quality, nan_value))

In [ ]:
import os
std_multiplier = 3
aspect_ratio = 0.7
colorbar_fraction = 0.05
colorbar_pad = 0.04


fig, axes = plt.subplots(1,2) 
ax1 = axes[0].imshow(
    np.ma.masked_equal(fh,nan_value), 
    cmap='YlGn',
    aspect=aspect_ratio, 
    extent=(lons.min(), lons.max(), lats.min(), lats.max()),
    vmin =  fh_avg - std_multiplier * fh_std,
    vmax =  fh_avg + std_multiplier * fh_std, 
    interpolation= "nearest"
)
axes[0].set_title('FH [m]')
axes[0].set_xlabel("Lon (wgs84)")
axes[0].set_ylabel("Lat (wgs84)")

ax2 = axes[1].imshow(
    np.ma.masked_equal(fh_quality, nan_value), 
    cmap='rainbow_r',
    aspect=aspect_ratio,
    extent=(lons.min(), lons.max(), lats.min(), lats.max()),
    vmin = 0,
    vmax = 100
)
axes[1].set_title('FH quality []')
axes[1].yaxis.set_tick_params(labelleft=False)
axes[1].set_xlabel("Lon (wgs84)")

fig.colorbar(ax1, ax=axes[0], orientation='vertical', fraction=colorbar_fraction, pad=colorbar_pad)
fig.colorbar(ax2, ax=axes[1], orientation='vertical', fraction=colorbar_fraction, pad=colorbar_pad)
fig.tight_layout()



plt.show()
plt.close

In [ ]:
#create an histogram
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# Istogramma per il file principale
axes[0].hist(np.ma.masked_equal(fh,nan_value).flatten(), bins=100, color='steelblue', alpha=0.8)
axes[0].set_title("L2A Measurement Histogram")
axes[0].set_xlabel("Value")
axes[0].set_ylabel("Frequency")
axes[0].grid(True)

# Istogramma per il file di qualità
axes[1].hist(np.ma.masked_equal(fh_quality, nan_value).flatten(), bins=100, color='darkorange', alpha=0.8)
axes[1].set_title("L2A Quality Histogram")
axes[1].set_xlabel("Value")
axes[1].set_ylabel("Frequency")
axes[1].grid(True)

plt.tight_layout()
plt.show()

#### Section 1.2: L2A FH Lut

In [ ]:
path_fh_lut =product_fh_l2a.annotation_lut_file

In [ ]:
lats = product_fh_l2a.Latitude
lons = product_fh_l2a.Longitude


#### Section 1.3: L2A FH Lut - FNF

In [ ]:
fnf_array =product_fh_l2a.FNF_FNF

In [ ]:
import matplotlib.colors as mcolors
import matplotlib.patches as mpatches
nan_value = 255
aspect_ratio =3


binary_cmap = mcolors.ListedColormap(['brown', 'green'])
white_patch = mpatches.Patch(color='brown', label='No Forest')
black_patch = mpatches.Patch(color='green', label='Forest')

fig, ax = plt.subplots(1, 1)
ax1 = ax.imshow(
    np.ma.masked_equal(fnf_array, nan_value), 
    cmap=binary_cmap,
    aspect=aspect_ratio, 
    extent=(lons.min(), lons.max(), lats.min(), lats.max()),
    vmin=0,
    vmax=1
)
ax.set_title('FNF')
ax.set_xlabel("Lon (wgs84)")
ax.set_ylabel("Lat (wgs84)")

fig.legend(handles=[white_patch, black_patch], loc='right', bbox_to_anchor=(1, 0.5))
fig.tight_layout()
plt.show()

## 🌳 Section 2: L2A FD

#### Section 2.1: L2A FD COG 

In [ ]:
path_cfm =  product_fd_l2a.measurement_cfm_file  
path_fd = product_fd_l2a.measurement_file   
path_prob = product_fd_l2a.measurement_probability_file   

plots

In [ ]:
# get the rasters

nan_value=product_fd_l2a.floatNoDataValue
#print(nan_value)
nan_value_int = 255

with rasterio.open(path_fd) as fd_raster:
    fd = fd_raster.read(1)
    
    # Get the coordinates for the plot
    transform = fd_raster.transform
    width = fd_raster.width
    height = fd_raster.height
    cols, rows = np.meshgrid(np.arange(width), np.arange(height))
    xs, ys = rasterio.transform.xy(transform, rows, cols)
    
    # Convert to numpy arrays
    lons = np.array(xs)
    lats = np.array(ys)



with rasterio.open(path_cfm) as cfm_raster:
    cfm = cfm_raster.read(1)

with rasterio.open(path_prob) as prob_raster:
    prob = prob_raster.read(1)



In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.colors as mcolors
import matplotlib.patches as mpatches

binary_cmap_cfm = mcolors.ListedColormap(['brown', 'green'])
binary_cmap_fd = mcolors.ListedColormap(['green', "brown"])

nan_value_float=product_fd_l2a.floatNoDataValue
#print(nan_value_float)
nan_value_int = 255
std_multiplier = 3
aspect_ratio = 1
colorbar_fraction = 0.05
colorbar_pad = 0.04

fig, axes = plt.subplots(1, 3, figsize=(10, 8))  # adjust plots scale and plot size if possible

cax1 = axes[0].imshow(
    np.ma.masked_equal(fd, nan_value_int),
    cmap=binary_cmap_fd,
    aspect=aspect_ratio,
    extent=(lons.min(), lons.max(), lats.min(), lats.max()),
    vmin=0,
    vmax=1)

axes[0].set_title('FD []')
axes[0].set_xlabel("Lon (wgs84)")
axes[0].set_ylabel("Lat (wgs84)")
# Adjust the colorbar labels for cax1
colorbar_cax1 = fig.colorbar(cax1, ax=axes[0], orientation='vertical', fraction=colorbar_fraction, pad=colorbar_pad)
colorbar_cax1.set_ticks([0, 1])
colorbar_cax1.set_ticklabels(['Not Disturbed', 'Disturbed'])

cax2 = axes[1].imshow(
    np.ma.masked_equal(cfm, nan_value_int),
    cmap=binary_cmap_cfm,
    aspect=aspect_ratio,
    extent=(lons.min(), lons.max(), lats.min(), lats.max()),
    vmin=0,
    vmax=1)

axes[1].yaxis.set_tick_params(labelleft=False)
axes[1].set_title('CFM []')
axes[1].set_xlabel("Lon (wgs84)")
# Adjust the colorbar labels for cax2
colorbar_cax2 = fig.colorbar(cax2, ax=axes[1], orientation='vertical', fraction=colorbar_fraction, pad=colorbar_pad)
colorbar_cax2.set_ticks([0, 1])
colorbar_cax2.set_ticklabels(['No Forest', 'Forest'])

masked_prob = np.ma.masked_equal(prob, nan_value_int)
vmin = masked_prob.min()
vmax = masked_prob.max()
cax3 = axes[2].imshow(
    masked_prob,
    cmap='viridis',
    aspect=aspect_ratio,
    extent=(lons.min(), lons.max(), lats.min(), lats.max()),
    vmin=vmin,
    vmax=vmax)

axes[2].yaxis.set_tick_params(labelleft=False)
axes[2].set_title('Prob of Change []')
axes[2].set_xlabel("Lon (wgs84)")


fig.colorbar(cax3, ax=axes[2], orientation='vertical', fraction=colorbar_fraction, pad=colorbar_pad)



fig.tight_layout()
plt.show()

#### Section 2.2:L2A FD Lut

In [ ]:
path_fd_lut =  product_fd_l2a.annotation_lut_file  

In [ ]:
lats = product_fd_l2a.Latitude
lons = product_fd_l2a.Longitude


#### Section 2.3: L2A FD LUT -  FNF

In [ ]:
fnf_array =product_fd_l2a.FNF_FNF

In [ ]:
import matplotlib.colors as mcolors
import matplotlib.patches as mpatches
nan_value = 255
aspect_ratio =3


binary_cmap = mcolors.ListedColormap(['brown', 'green'])
white_patch = mpatches.Patch(color='brown', label='No Forest')
black_patch = mpatches.Patch(color='green', label='Forest')

fig, ax = plt.subplots(1, 1)
ax1 = ax.imshow(
    np.ma.masked_equal(fnf_array, nan_value), 
    cmap=binary_cmap,
    aspect=aspect_ratio, 
    extent=(lons.min(), lons.max(), lats.min(), lats.max()),
    vmin=0,
    vmax=1
)
ax.set_title('FNF')
ax.set_xlabel("Lon (wgs84)")
ax.set_ylabel("Lat (wgs84)")

fig.legend(handles=[white_patch, black_patch], loc='right', bbox_to_anchor=(1, 0.5))
fig.tight_layout()
plt.show()

#### Section 2.4: L2A FD  LUT -  ACM

In [ ]:

layer1 =product_fd_l2a.ACM_layer1
layer2 =product_fd_l2a.ACM_layer2
layer3 =product_fd_l2a.ACM_layer3
layer4 =product_fd_l2a.ACM_layer4
layer5 =product_fd_l2a.ACM_layer5
layer6 =product_fd_l2a.ACM_layer6
layer7 =product_fd_l2a.ACM_layer7
layer8 =product_fd_l2a.ACM_layer8
layer9 =product_fd_l2a.ACM_layer9

In [ ]:
nan_value_float = -9999
nan_value_int = 255
std_multiplier = 3
aspect_ratio = 3
colorbar_fraction = 0.05
colorbar_pad = 0.04

# abs
entry11 = layer1
entry12 = layer2
entry13 = layer4

entry21 = layer2
entry22 = layer6
entry23 = layer7

entry31 = layer4
entry32 = layer7
entry33 = layer9

def mean_std_masked(array, nodata_value):
    masked = np.ma.masked_equal(array, nodata_value)
    return masked.mean(), masked.std()

entry11_avg, entry11_std = mean_std_masked((entry11), nan_value_float)
entry12_avg, entry12_std = mean_std_masked((entry12), nan_value_float)
entry13_avg, entry13_std = mean_std_masked((entry13), nan_value_float)
entry21_avg, entry21_std = mean_std_masked((entry21), nan_value_float)
entry22_avg, entry22_std = mean_std_masked((entry22), nan_value_float)
entry23_avg, entry23_std = mean_std_masked((entry23), nan_value_float)
entry31_avg, entry31_std = mean_std_masked((entry31), nan_value_float)
entry32_avg, entry32_std = mean_std_masked((entry32), nan_value_float)
entry33_avg, entry33_std = mean_std_masked((entry33), nan_value_float)


fig, axes = plt.subplots(3,3, figsize=(12,12)) 

ax1 = axes[0,0].imshow( 
    entry11, 
    cmap='rainbow',
    aspect=aspect_ratio,
    extent=(lons.min(), lons.max(), lats.min(), lats.max()),
    vmin =  entry11_avg - std_multiplier * entry11_std,
    vmax =  entry11_avg + std_multiplier * entry11_std
)
axes[0,0].set_title('Amp [HH-HH]')
axes[0,0].xaxis.set_tick_params(labelleft=False)
axes[0,0].xaxis.set_tick_params(labelleft=False)
axes[0,0].set_ylabel("Lat (wgs84)")


ax2 = axes[0,1].imshow( 
    entry12, 
    cmap='rainbow',
    aspect=aspect_ratio,
    extent=(lons.min(), lons.max(), lats.min(), lats.max()),
    vmin =  entry12_avg - std_multiplier * entry12_std,
    vmax =  entry12_avg + std_multiplier * entry12_std
)
axes[0,1].set_title('Amp [HH-XP]')
axes[0,1].yaxis.set_tick_params(labelleft=False)
axes[0,1].xaxis.set_tick_params(labelleft=False)
axes[0,1].set_ylabel("Lat (wgs84)")

ax3 = axes[0,2].imshow( 
    entry13, 
    cmap='rainbow',
    aspect=aspect_ratio,
    extent=(lons.min(), lons.max(), lats.min(), lats.max()),
    vmin =  entry13_avg - std_multiplier * entry13_std,
    vmax =  entry13_avg + std_multiplier * entry13_std
)
axes[0,2].set_title('Amp [HH-VV]')
axes[0,2].xaxis.set_tick_params(labelleft=False)
axes[0,2].xaxis.set_tick_params(labelleft=False)
axes[0,2].set_ylabel("Lat (wgs84)")

ax4 = axes[1,0].imshow( 
    entry21, 
    cmap='rainbow',
    aspect=aspect_ratio,
    extent=(lons.min(), lons.max(), lats.min(), lats.max()),
    vmin =  entry21_avg - std_multiplier * entry21_std,
    vmax =  entry21_avg + std_multiplier * entry21_std
)
axes[1,0].set_title('Amp [HH-XP]')
axes[1,0].yaxis.set_tick_params(labelleft=False)
axes[1,0].xaxis.set_tick_params(labelleft=False)
axes[1,0].set_ylabel("Lat (wgs84)")

ax5 = axes[1,1].imshow( 
    entry22, 
    cmap='rainbow',
    aspect=aspect_ratio,
    extent=(lons.min(), lons.max(), lats.min(), lats.max()),
    vmin =  entry22_avg - std_multiplier * entry22_std,
    vmax =  entry22_avg + std_multiplier * entry22_std
)
axes[1,1].set_title('Amp [XP-XP]')
axes[1,1].yaxis.set_tick_params(labelleft=False)
axes[1,1].xaxis.set_tick_params(labelleft=False)
axes[1,1].set_ylabel("Lat (wgs84)")

ax6 = axes[1,2].imshow( 
    entry23, 
    cmap='rainbow',
    aspect=aspect_ratio,
    extent=(lons.min(), lons.max(), lats.min(), lats.max()),
    vmin =  entry23_avg - std_multiplier * entry23_std,
    vmax =  entry23_avg + std_multiplier * entry23_std
)
axes[1,2].set_title('Amp [XP-VV]')
axes[1,2].yaxis.set_tick_params(labelleft=False)
axes[1,2].xaxis.set_tick_params(labelleft=False)
axes[1,2].set_xlabel("Lon (wgs84)")

ax7 = axes[2,0].imshow( 
    entry31, 
    cmap='rainbow',
    aspect=aspect_ratio,
    extent=(lons.min(), lons.max(), lats.min(), lats.max()),
    vmin =  entry31_avg - std_multiplier * entry31_std,
    vmax =  entry31_avg + std_multiplier * entry31_std
)
axes[2,0].set_title('Amp [HH-VV]')
axes[2,0].yaxis.set_tick_params(labelleft=False)
axes[2,0].xaxis.set_tick_params(labelleft=False)
axes[2,0].set_xlabel("Lon (wgs84)")

ax8 = axes[2,1].imshow( 
    entry32, 
    cmap='rainbow',
    aspect=aspect_ratio,
    extent=(lons.min(), lons.max(), lats.min(), lats.max()),
    vmin =  entry32_avg - std_multiplier * entry32_std,
    vmax =  entry32_avg + std_multiplier * entry32_std
)
axes[2,1].set_title('Amp [XP-VV]')
axes[2,1].yaxis.set_tick_params(labelleft=False)
axes[2,1].xaxis.set_tick_params(labelleft=False)
axes[2,1].set_xlabel("Lon (wgs84)")


ax9 = axes[2,2].imshow( 
    entry33, 
    cmap='rainbow',
    aspect=aspect_ratio,
    extent=(lons.min(), lons.max(), lats.min(), lats.max()),
    vmin =  entry33_avg - std_multiplier * entry33_std,
    vmax =  entry33_avg + std_multiplier * entry33_std
)
axes[2,2].set_title('Amp [VV-VV]')
axes[2,2].yaxis.set_tick_params(labelleft=False)
axes[2,2].xaxis.set_tick_params(labelleft=False)
axes[2,2].set_xlabel("Lon (wgs84)")

fig.colorbar(ax1, ax=axes[0,0], orientation='vertical', fraction=colorbar_fraction, pad=colorbar_pad)
fig.colorbar(ax2, ax=axes[0,1], orientation='vertical', fraction=colorbar_fraction, pad=colorbar_pad)
fig.colorbar(ax3, ax=axes[0,2], orientation='vertical', fraction=colorbar_fraction, pad=colorbar_pad)
fig.colorbar(ax4, ax=axes[1,0], orientation='vertical', fraction=colorbar_fraction, pad=colorbar_pad)
fig.colorbar(ax5, ax=axes[1,1], orientation='vertical', fraction=colorbar_fraction, pad=colorbar_pad)
fig.colorbar(ax6, ax=axes[1,2], orientation='vertical', fraction=colorbar_fraction, pad=colorbar_pad)
fig.colorbar(ax7, ax=axes[2,0], orientation='vertical', fraction=colorbar_fraction, pad=colorbar_pad)
fig.colorbar(ax8, ax=axes[2,1], orientation='vertical', fraction=colorbar_fraction, pad=colorbar_pad)
fig.colorbar(ax9, ax=axes[2,2], orientation='vertical', fraction=colorbar_fraction, pad=colorbar_pad)

fig.tight_layout()

In [ ]:
nan_value_float=product_fd_l2a.floatNoDataValue
#print(nan_value_float)

nan_value_int = 255
std_multiplier = 3
aspect_ratio = 3
colorbar_fraction = 0.05
colorbar_pad = 0.04

def mean_std_masked(array, nodata_value):
    masked = np.ma.masked_equal(array, nodata_value)
    return masked.mean(), masked.std()

layer1_avg, layer1_std = mean_std_masked(layer1, nan_value_float)
layer2_avg, layer2_std = mean_std_masked(layer2, nan_value_float)
layer3_avg, layer3_std = mean_std_masked(layer3, nan_value_float)
layer4_avg, layer4_std = mean_std_masked(layer4, nan_value_float)
layer5_avg, layer5_std = mean_std_masked(layer5, nan_value_float)
layer6_avg, layer6_std = mean_std_masked(layer6, nan_value_float)
layer7_avg, layer7_std = mean_std_masked(layer7, nan_value_float)
layer8_avg, layer8_std = mean_std_masked(layer8, nan_value_float)
layer9_avg, layer9_std = mean_std_masked(layer9, nan_value_float)


fig, axes = plt.subplots(3,3, figsize=(12,12)) 

ax1 = axes[0,0].imshow( 
    layer1, 
    cmap='rainbow',
    aspect=aspect_ratio,
    extent=(lons.min(), lons.max(), lats.min(), lats.max()),
    vmin =  layer1_avg - std_multiplier * layer1_std,
    vmax =  layer1_avg + std_multiplier * layer1_std
)
axes[0,0].set_title('layer1 [TBD]')
axes[0,0].xaxis.set_tick_params(labelleft=False)
axes[0,0].set_ylabel("Lat (wgs84)")


ax2 = axes[0,1].imshow( 
    layer2, 
    cmap='rainbow',
    aspect=aspect_ratio,
    extent=(lons.min(), lons.max(), lats.min(), lats.max()),
    vmin =  layer2_avg - std_multiplier * layer2_std,
    vmax =  layer2_avg + std_multiplier * layer2_std
)
axes[0,1].set_title('layer2 [TBD]')
axes[0,1].yaxis.set_tick_params(labelleft=False)
axes[0,1].xaxis.set_tick_params(labelleft=False)


ax3 = axes[0,2].imshow( 
    layer3, 
    cmap='rainbow',
    aspect=aspect_ratio,
    extent=(lons.min(), lons.max(), lats.min(), lats.max()),
    vmin =  layer3_avg - std_multiplier * layer3_std,
    vmax =  layer3_avg + std_multiplier * layer3_std
)
axes[0,2].set_title('layer3 [TBD]')
axes[0,2].yaxis.set_tick_params(labelleft=False)
axes[0,2].xaxis.set_tick_params(labelleft=False)


ax4 = axes[1,0].imshow( 
    layer4, 
    cmap='rainbow',
    aspect=aspect_ratio,
    extent=(lons.min(), lons.max(), lats.min(), lats.max()),
    vmin =  layer4_avg - std_multiplier * layer4_std,
    vmax =  layer4_avg + std_multiplier * layer4_std
)
axes[1,0].set_title('layer4 [TBD]')
axes[1,0].xaxis.set_tick_params(labelleft=False)
axes[1,0].set_ylabel("Lat (wgs84)")


ax5 = axes[1,1].imshow( 
    layer5, 
    cmap='rainbow',
    aspect=aspect_ratio,
    extent=(lons.min(), lons.max(), lats.min(), lats.max()),
    vmin =  layer5_avg - std_multiplier * layer5_std,
    vmax =  layer5_avg + std_multiplier * layer5_std
)
axes[1,1].set_title('layer5 [TBD]')
axes[1,1].yaxis.set_tick_params(labelleft=False)
axes[1,1].xaxis.set_tick_params(labelleft=False)


ax6 = axes[1,2].imshow( 
    layer6, 
    cmap='rainbow',
    aspect=aspect_ratio,
    extent=(lons.min(), lons.max(), lats.min(), lats.max()),
    vmin =  layer6_avg - std_multiplier * layer6_std,
    vmax =  layer6_avg + std_multiplier * layer6_std
)
axes[1,2].set_title('layer6 [TND]')
axes[1,2].yaxis.set_tick_params(labelleft=False)
axes[1,2].xaxis.set_tick_params(labelleft=False)


ax7 = axes[2,0].imshow( 
    layer7, 
    cmap='rainbow',
    aspect=aspect_ratio,
    extent=(lons.min(), lons.max(), lats.min(), lats.max()),
    vmin =  layer7_avg - std_multiplier * layer7_std,
    vmax =  layer7_avg + std_multiplier * layer7_std
)
axes[2,0].set_title('layer7 [TBD]')
axes[2,0].set_xlabel("Lon (wgs84)")
axes[2,0].set_ylabel("Lat (wgs84)")


ax8 = axes[2,1].imshow( 
    layer8, 
    cmap='rainbow',
    aspect=aspect_ratio,
    extent=(lons.min(), lons.max(), lats.min(), lats.max()),
    vmin =  layer8_avg - std_multiplier * layer8_std,
    vmax =  layer8_avg + std_multiplier * layer8_std
)
axes[2,1].set_title('layer8 [TBD]')
axes[2,1].yaxis.set_tick_params(labelleft=False)
axes[2,1].set_xlabel("Lon (wgs84)")


ax9 = axes[2,2].imshow( 
    layer9, 
    cmap='rainbow',
    aspect=aspect_ratio,
    extent=(lons.min(), lons.max(), lats.min(), lats.max()),
    vmin =  layer9_avg - std_multiplier * layer9_std,
    vmax =  layer9_avg + std_multiplier * layer9_std
)
axes[2,2].set_title('layer9 [TBD]')
axes[2,2].yaxis.set_tick_params(labelleft=False)
axes[2,2].set_xlabel("Lon (wgs84)")

fig.colorbar(ax1, ax=axes[0,0], orientation='vertical', fraction=colorbar_fraction, pad=colorbar_pad)
fig.colorbar(ax2, ax=axes[0,1], orientation='vertical', fraction=colorbar_fraction, pad=colorbar_pad)
fig.colorbar(ax3, ax=axes[0,2], orientation='vertical', fraction=colorbar_fraction, pad=colorbar_pad)
fig.colorbar(ax4, ax=axes[1,0], orientation='vertical', fraction=colorbar_fraction, pad=colorbar_pad)
fig.colorbar(ax5, ax=axes[1,1], orientation='vertical', fraction=colorbar_fraction, pad=colorbar_pad)
fig.colorbar(ax6, ax=axes[1,2], orientation='vertical', fraction=colorbar_fraction, pad=colorbar_pad)
fig.colorbar(ax7, ax=axes[2,0], orientation='vertical', fraction=colorbar_fraction, pad=colorbar_pad)
fig.colorbar(ax8, ax=axes[2,1], orientation='vertical', fraction=colorbar_fraction, pad=colorbar_pad)
fig.colorbar(ax9, ax=axes[2,2], orientation='vertical', fraction=colorbar_fraction, pad=colorbar_pad)

fig.tight_layout()
plt.show()

#### Section 2.5: L2A FD LUT -  numberOfAverages

In [ ]:
numberOfAverages_array = product_fd_l2a.numberOfAverages_numberOfAverages

In [ ]:
numberOfAverages_array

In [ ]:
nan_value = 255
std_multiplier = 3
aspect_ratio =3
colorbar_fraction = 0.05
colorbar_pad = 0.04

fig, ax = plt.subplots(1, 1)

masked = np.ma.masked_equal(numberOfAverages_array, nan_value)

# Calcola vmin e vmax escludendo gli outlier
vmin = np.percentile(masked.compressed(), 2)
vmax = np.percentile(masked.compressed(), 98)

ax1 = ax.imshow(
    masked, 
    cmap="rainbow",
    aspect=aspect_ratio, 
    extent=(lons.min(), lons.max(), lats.min(), lats.max()),
    vmin=vmin,
    vmax=vmax
)
ax.set_title('numberOfAverages')
ax.set_xlabel("Lon (wgs84)")
ax.set_ylabel("Lat (wgs84)")

fig.colorbar(ax1, ax=ax, orientation='vertical', fraction=colorbar_fraction, pad=colorbar_pad)
fig.tight_layout()
plt.show()

## 🍃 Section 3: L2A GN

#### Section 3.1: L2A GN COG 

In [ ]:
path_gn = product_gn_l2a.measurement_file 

In [ ]:
nan_value_float=product_gn_l2a.floatNoDataValue

nan_value_int = 255

with rasterio.open(path_gn) as gn_raster:
    sigma_hh = gn_raster.read(1) #nupy.ndarray object
    sigma_hh = np.ma.masked_equal(sigma_hh, nan_value)
        
    sigma_xx = gn_raster.read(2) #nupy.ndarray object
    sigma_xx = np.ma.masked_equal(sigma_xx, nan_value)

    
    sigma_vv = gn_raster.read(3) #nupy.ndarray object
    sigma_vv = np.ma.masked_equal(sigma_vv, nan_value)

   
    # Get the coordinates for the plot
    transform = gn_raster.transform
    width = gn_raster.width
    height = gn_raster.height
    cols, rows = np.meshgrid(np.arange(width), np.arange(height))
    xs, ys = rasterio.transform.xy(transform, rows, cols)
    
    # Convert to numpy arrays
    lons = np.array(xs)
    lats = np.array(ys)

In [ ]:

sigma_hh_safe = np.where(sigma_hh > 0, sigma_hh, np.nan)
sigma_hh_db = 20*np.log10(sigma_hh_safe)
sigma_hh_db_avg = np.nanmean(sigma_hh_db)
sigma_hh_db_std = np.nanstd(sigma_hh_db)

sigma_xx_safe=np.where(sigma_xx > 0, sigma_xx, np.nan)
sigma_xx_db = 20*np.log10(sigma_xx_safe)
sigma_xx_db_avg = np.nanmean(sigma_xx_db)
sigma_xx_db_std = np.nanstd(sigma_xx_db)

sigma_vv_safe=np.where(sigma_vv > 0, sigma_vv, np.nan)
sigma_vv_db = 20*np.log10(sigma_vv_safe)
sigma_vv_db_avg = np.nanmean(sigma_vv_db)
sigma_vv_db_std = np.nanstd(sigma_vv_db)



In [ ]:
## plot 

std_multiplier = 1
aspect_ratio = 1
colorbar_fraction = 0.05
colorbar_pad = 0.04

fig, axes = plt.subplots(1,3, figsize=(8, 5)) # adjust plots scale and plot size if possibile

cax1 = axes[0].imshow( 
    sigma_hh_db, 
    cmap='rainbow',
    aspect= aspect_ratio , 
    extent=(lons.min(), lons.max(), lats.min(), lats.max()),
    vmin = sigma_hh_db_avg - std_multiplier * sigma_hh_db_std, 
    vmax = sigma_hh_db_avg + std_multiplier * sigma_hh_db_std ) 

axes[0].set_title('db sigma HH')
axes[0].set_xlabel("Lon (wgs84)")
axes[0].set_ylabel("Lat (wgs84)")

cax2 = axes[1].imshow( 
    sigma_xx_db, 
    cmap='rainbow',
    aspect= aspect_ratio , 
    extent=(lons.min(), lons.max(), lats.min(), lats.max()),
    vmin = sigma_xx_db_avg - std_multiplier * sigma_xx_db_std, 
    vmax = sigma_xx_db_avg + std_multiplier * sigma_xx_db_std )
axes[1].set_title('db sigma XX')
axes[1].yaxis.set_tick_params(labelleft=False)
axes[1].set_xlabel("Lon (wgs84)")

cax3 = axes[2].imshow( 
    sigma_vv_db, 
    cmap='rainbow',
    aspect= aspect_ratio , 
    extent=(lons.min(), lons.max(), lats.min(), lats.max()),
    vmin = sigma_vv_db_avg - std_multiplier * sigma_vv_db_std, 
    vmax = sigma_vv_db_avg + std_multiplier * sigma_vv_db_std )
axes[2].set_title('db sigma VV')
axes[2].yaxis.set_tick_params(labelleft=False)
axes[2].set_xlabel("Lon (wgs84)")

fig.colorbar(cax1, ax=axes[0], orientation='vertical', fraction=colorbar_fraction, pad=colorbar_pad) 
fig.colorbar(cax2, ax=axes[1], orientation='vertical', fraction=colorbar_fraction, pad=colorbar_pad)
fig.colorbar(cax3, ax=axes[2], orientation='vertical', fraction=colorbar_fraction, pad=colorbar_pad)
fig.tight_layout()
plt.show()

#### Section 3.2: L2A GN LUT

In [ ]:
path_gn_lut = product_gn_l2a.annotation_lut_file 

In [ ]:
lats = product_gn_l2a.Latitude
lons = product_gn_l2a.Longitude


#### Section 3.3: L2A GN LUT -  FNF

In [ ]:
#fnf = lut_gn.groups["FNF"]

In [ ]:
fnf_array = product_gn_l2a.FNF_FNF

In [ ]:
import matplotlib.colors as mcolors
import matplotlib.patches as mpatches
nan_value = 255

aspect_ratio =3


binary_cmap = mcolors.ListedColormap(['brown', 'green'])
white_patch = mpatches.Patch(color='brown', label='No Forest')
black_patch = mpatches.Patch(color='green', label='Forest')

fig, ax = plt.subplots(1, 1)
ax1 = ax.imshow(
    np.ma.masked_equal(fnf_array, nan_value), 
    cmap=binary_cmap,
    aspect=aspect_ratio, 
    extent=(lons.min(), lons.max(), lats.min(), lats.max()),
    vmin=0,
    vmax=1
)
ax.set_title('FNF')
ax.set_xlabel("Lon (wgs84)")
ax.set_ylabel("Lat (wgs84)")

fig.legend(handles=[white_patch, black_patch], loc='right', bbox_to_anchor=(1, 0.5))
fig.tight_layout()
plt.show()

#### Section 3.4: L2A GN LUT -  localIncidenceAngle

In [ ]:
#localIncidenceAngle = lut_gn.groups["localIncidenceAngle"]
#localIncidenceAngle
localIncidenceAngle_array = product_gn_l2a.localIncidenceAngle_localIncidenceAngle

In [ ]:

nan_value=product_gn_l2a.floatNoDataValue


localIncidenceAngle_array_avg = np.mean(np.ma.masked_equal(localIncidenceAngle_array, nan_value))
localIncidenceAngle_array_std = np.std(np.ma.masked_equal(localIncidenceAngle_array, nan_value))
localIncidenceAngle_array_max = np.max(np.ma.masked_equal(localIncidenceAngle_array, nan_value))
localIncidenceAngle_array_min = np.min(np.ma.masked_equal(localIncidenceAngle_array, nan_value))

std_multiplier = 3
aspect_ratio =3
colorbar_fraction = 0.05
colorbar_pad = 0.04

fig, (ax, ax2) = plt.subplots(1, 2, figsize=(10, 5), gridspec_kw={"width_ratios": [1, 0.3]})

ax1 = ax.imshow(
    np.ma.masked_equal(localIncidenceAngle_array, nan_value), 
    cmap="rainbow",
    aspect=aspect_ratio, 
    extent=(lons.min(), lons.max(), lats.min(), lats.max()),
    vmin =  localIncidenceAngle_array_avg - std_multiplier * localIncidenceAngle_array_std,
    vmax =  localIncidenceAngle_array_avg + std_multiplier * localIncidenceAngle_array_std
)
ax.set_title('localIncidenceAngle_array [deg]')
ax.set_xlabel("Lon (wgs84)")
ax.set_ylabel("Lat (wgs84)")
ax2.axis("off")
stats_text = (
    f"Mean:\n{localIncidenceAngle_array_avg:.2f}°\n\n"
    f"Std Dev:\n{localIncidenceAngle_array_std:.2f}°\n\n"
    f"Min:\n{localIncidenceAngle_array_min:.2f}°\n\n"
    f"Max:\n{localIncidenceAngle_array_max:.2f}°"
)
ax2.text(0, 1, stats_text, fontsize=11, va='top')
fig.colorbar(ax1, ax=ax, orientation='vertical', fraction=colorbar_fraction, pad=colorbar_pad)
fig.tight_layout()
plt.show()

## 🌲 Section 4: L2B FH

#### Section 4.1: L2B FH  COG 

In [ ]:
import rasterio
import netCDF4
import numpy as np
import matplotlib.pyplot as plt
import warnings
import BiomassProduct 
from pathlib import Path
warnings.filterwarnings('ignore', category=DeprecationWarning)

folder_TDS = Path("..") / "Muriel_test/L2B"
paths = BiomassProduct.L2BProductPaths(folder_TDS)     
    
product_fh_l2b = BiomassProduct.l2b_fh(paths.fh)
product_fd_l2b = BiomassProduct.l2b_fd(paths.fd)
product_agb_l2b = BiomassProduct.l2b_agb(paths.agb)

In [ ]:
path_fh = product_fh_l2b.measurement_file   
path_fh_quality = product_fh_l2b.measurement_fhquality_file 

print(path_fh)
print (path_fh_quality)

In [ ]:
nan_value=product_fd_l2b.floatNoDataValue

with rasterio.open(path_fh) as fh_raster:
    fh = fh_raster.read(1)
    
    # Get the coordinates for the plot
    transform = fh_raster.transform
    width = fh_raster.width
    height = fh_raster.height
    cols, rows = np.meshgrid(np.arange(width), np.arange(height))
    xs, ys = rasterio.transform.xy(transform, rows, cols)
    
    # Convert to numpy arrays
    lons = np.array(xs)
    lats = np.array(ys)



with rasterio.open(path_fh_quality) as fh_quality_raster:
    fh_quality = fh_quality_raster.read(1)

fh_avg = np.mean(np.ma.masked_equal(fh,nan_value))
fh_std = np.std(np.ma.masked_equal(fh, nan_value))

fh_quality_avg = np.mean(np.ma.masked_equal(fh_quality, nan_value))
fh_quality_std = np.std(np.ma.masked_equal(fh_quality, nan_value))

In [ ]:
std_multiplier = 3
aspect_ratio = 1
colorbar_fraction = 0.05
colorbar_pad = 0.04
nan_value=product_fh_l2b.floatNoDataValue

fig, axes = plt.subplots(1,2, figsize=(8,8)) 
ax1 = axes[0].imshow(
    np.ma.masked_equal(fh,nan_value), 
    cmap='YlGn',
    aspect=aspect_ratio, 
    extent=(lons.min(), lons.max(), lats.min(), lats.max()),
    vmin =  fh_avg - std_multiplier * fh_std,
    vmax =  fh_avg + std_multiplier * fh_std
)
axes[0].set_title('FH [m]')
axes[0].set_xlabel("Lon (wgs84)")
axes[0].set_ylabel("Lat (wgs84)")

ax2 = axes[1].imshow(
    np.ma.masked_equal(fh_quality, nan_value), 
    cmap='rainbow_r',
    aspect=aspect_ratio,
    extent=(lons.min(), lons.max(), lats.min(), lats.max()),
    vmin = 0,
    vmax = 100
)
axes[1].set_title('FH quality []')
axes[1].yaxis.set_tick_params(labelleft=False)
axes[1].set_xlabel("Lon (wgs84)")

fig.colorbar(ax1, ax=axes[0], orientation='vertical', fraction=colorbar_fraction, pad=colorbar_pad)
fig.colorbar(ax2, ax=axes[1], orientation='vertical', fraction=colorbar_fraction, pad=colorbar_pad)
fig.tight_layout()
plt.show()

#### Section 4.2: L2B FH  Annotation folder

In [ ]:
acquisition_id_image_path = product_fh_l2b.annotation_acquisition_file 
fnf_path = product_fh_l2b.annotation_bps_fnf_file  
heatmap_path = product_fh_l2b.annotation_heatmap_file 

#### Section 4.3: L2B FH  Annotation folder - FNF

In [ ]:
with rasterio.open(fnf_path) as fnf_raster:
    fnf_array = fnf_raster.read(1)
    
    # Get the coordinates for the plot
    transform = fnf_raster.transform
    width = fnf_raster.width
    height = fnf_raster.height
    cols, rows = np.meshgrid(np.arange(width), np.arange(height))
    xs, ys = rasterio.transform.xy(transform, rows, cols)
    
    # Convert to numpy arrays
    lons = np.array(xs)
    lats = np.array(ys)

In [ ]:
import matplotlib.colors as mcolors
import matplotlib.patches as mpatches
nan_value = 255
aspect_ratio =1


binary_cmap = mcolors.ListedColormap(['brown', 'green'])
white_patch = mpatches.Patch(color='brown', label='No Forest')
black_patch = mpatches.Patch(color='green', label='Forest')

fig, ax = plt.subplots(1, 1)
ax1 = ax.imshow(
    np.ma.masked_equal(fnf_array, nan_value), 
    cmap=binary_cmap,
    aspect=aspect_ratio, 
    extent=(lons.min(), lons.max(), lats.min(), lats.max()),
    vmin=0,
    vmax=1
)
ax.set_title('FNF')
ax.set_xlabel("Lon (wgs84)")
ax.set_ylabel("Lat (wgs84)")

fig.legend(handles=[white_patch, black_patch], loc='right', bbox_to_anchor=(1, 0.5))
fig.tight_layout()
plt.show()

#### Section 4.4: L2B FH  Annotation folder - HeatMap

In [ ]:
with rasterio.open(heatmap_path) as heatmap_raster:
    weight_sum_array = heatmap_raster.read(1)
    
    
    # Get the coordinates for the plot
    transform = heatmap_raster.transform
    width = heatmap_raster.width
    height = heatmap_raster.height
    cols, rows = np.meshgrid(np.arange(width), np.arange(height))
    xs, ys = rasterio.transform.xy(transform, rows, cols)
    
    # Convert to numpy arrays
    lons = np.array(xs)
    lats = np.array(ys)

In [ ]:
nan_value=product_fh_l2b.floatNoDataValue
std_multiplier = 3
aspect_ratio =1
colorbar_fraction = 0.05
colorbar_pad = 0.04


weight_sum_array_avg = np.mean(np.ma.masked_equal(weight_sum_array,nan_value))
weight_sum_array_std = np.std(np.ma.masked_equal(weight_sum_array, nan_value))



fig, ax = plt.subplots(1, 1)
ax1 = ax.imshow(
    np.ma.masked_equal(weight_sum_array, nan_value), 
    cmap="rainbow",
    aspect=aspect_ratio, 
    extent=(lons.min(), lons.max(), lats.min(), lats.max()),
    vmin=0,
    vmax=weight_sum_array_avg  + std_multiplier*weight_sum_array_std
)
ax.set_title('Heatmap \n weight sum')
ax.set_xlabel("Lon (wgs84)")
ax.set_ylabel("Lat (wgs84)")

fig.colorbar(ax1, ax=ax, orientation='vertical', fraction=colorbar_fraction, pad=colorbar_pad)
fig.tight_layout()
plt.show()

In [ ]:
nan_value = 0  
std_multiplier = 3
aspect_ratio =1
colorbar_fraction = 0.05
colorbar_pad = 0.04


weight_sum_array_avg = np.mean(np.ma.masked_equal(weight_sum_array,nan_value))
weight_sum_array_std = np.std(np.ma.masked_equal(weight_sum_array, nan_value))



fig, ax = plt.subplots(1, 1)
ax1 = ax.imshow(
    np.ma.masked_equal(weight_sum_array, nan_value), 
    cmap="rainbow",
    aspect=aspect_ratio, 
    extent=(lons.min(), lons.max(), lats.min(), lats.max()),
    vmin=0,
    vmax=weight_sum_array_avg  + std_multiplier*weight_sum_array_std
)
ax.set_title('Heatmap \n weight sum')
ax.set_xlabel("Lon (wgs84)")
ax.set_ylabel("Lat (wgs84)")

fig.colorbar(ax1, ax=ax, orientation='vertical', fraction=colorbar_fraction, pad=colorbar_pad)
fig.tight_layout()
plt.show()

In [ ]:
"""
no data are mixed with zero....
"""

nan_value_list = [nan_value, 0]
std_multiplier = 3
aspect_ratio =1
colorbar_fraction = 0.05
colorbar_pad = 0.04

weight_sum_array = np.where(
    (weight_sum_array == nan_value_list[0]) | (weight_sum_array == nan_value_list[1]),
    np.nan,
    weight_sum_array)


weight_sum_array_avg = np.nanmean(weight_sum_array)
weight_sum_array_std = np.nanstd(weight_sum_array)



fig, ax = plt.subplots(1, 1)
ax1 = ax.imshow(
    weight_sum_array, 
    cmap="rainbow",
    aspect=aspect_ratio, 
    extent=(lons.min(), lons.max(), lats.min(), lats.max()),
    vmin=0,
    vmax=weight_sum_array_avg  + std_multiplier*weight_sum_array_std
)
ax.set_title('Heatmap \n weight sum')
ax.set_xlabel("Lon (wgs84)")
ax.set_ylabel("Lat (wgs84)")

fig.colorbar(ax1, ax=ax, orientation='vertical', fraction=colorbar_fraction, pad=colorbar_pad)
fig.tight_layout()
plt.show()

#### Section 4.5: L2B FH  Annotation folder - Acquisition ID Image

In [ ]:
acquisition_id_image_layers = {}

with rasterio.open(acquisition_id_image_path) as acquisition_id_image:
    num_channels = acquisition_id_image.count

    for i in range(0, num_channels):
        acquisition_id_image_layers[str(i + 1)] = acquisition_id_image.read(i+1)    
    
    
    # Get the coordinates for the plot
    transform = acquisition_id_image.transform
    width = acquisition_id_image.width
    height = acquisition_id_image.height
    cols, rows = np.meshgrid(np.arange(width), np.arange(height))
    xs, ys = rasterio.transform.xy(transform, rows, cols)
    
    # Convert to numpy arrays
    lons = np.array(xs)
    lats = np.array(ys)

In [ ]:
"""
problem with nan value
"""

nan_value = 0
aspect_ratio =1
colorbar_fraction = 0.05
colorbar_pad = 0.04

fig, axes = plt.subplots(num_channels, 1, figsize=(10, 5 * num_channels))

for i in range(1, num_channels + 1):
    data = acquisition_id_image_layers[str(i)]
    
    # Calculate the aspect ratio
    aspect_ratio = (lons.max() - lons.min()) / (lats.max() - lats.min())
    
    # Plot the data
    ax = axes[i - 1] if num_channels > 1 else axes
    ax1 = ax.imshow(
        np.ma.masked_equal(data, nan_value), 
        cmap="rainbow",
        aspect=aspect_ratio, 
        extent=(lons.min(), lons.max(), lats.min(), lats.max()),
        vmin=0,
        vmax=np.ma.masked_equal(data, nan_value).max()  # Example of using mean and std for vmin and vmax
    )
    ax.set_title(f'Acquisition ID {i}')
    ax.set_xlabel("Lon (wgs84)")
    ax.set_ylabel("Lat (wgs84)")
    
    # Add colorbar
    fig.colorbar(ax1, ax=ax, orientation='vertical', fraction=colorbar_fraction, pad=colorbar_pad)

fig.tight_layout()
plt.show()

## 🌴 Section 5: L2B FD

#### Section 5.1: L2B FD  COG 

In [ ]:
path_cfm =  product_fd_l2b.measurement_cfm_file  
path_fd =   product_fd_l2b.measurement_file  
path_prob = product_fd_l2b.measurement_probability_file

print(path_cfm)
print(path_fd)
print(path_prob)

In [ ]:
# get the rasters
nan_value_float=product_fd_l2b.floatNoDataValue
nan_value_int = 255

with rasterio.open(path_fd) as fd_raster:
    fd = fd_raster.read(1)
    
    # Get the coordinates for the plot
    transform = fd_raster.transform
    width = fd_raster.width
    height = fd_raster.height
    cols, rows = np.meshgrid(np.arange(width), np.arange(height))
    xs, ys = rasterio.transform.xy(transform, rows, cols)
    
    # Convert to numpy arrays
    lons = np.array(xs)
    lats = np.array(ys)



with rasterio.open(path_cfm) as cfm_raster:
    cfm = cfm_raster.read(1)

with rasterio.open(path_prob) as prob_raster:
    prob = prob_raster.read(1)


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.colors as mcolors
import matplotlib.patches as mpatches

binary_cmap_cfm = mcolors.ListedColormap(['brown', 'green'])


binary_cmap_fd = mcolors.ListedColormap(['green', "brown"])

nan_value_float=product_fd_l2b.floatNoDataValue

nan_value_int = 255
std_multiplier = 3
aspect_ratio = 1
colorbar_fraction = 0.05
colorbar_pad = 0.04

fig, axes = plt.subplots(3,1, figsize=(7, 15))  # adjust plots scale and plot size if possible

cax1 = axes[0].imshow(
    np.ma.masked_equal(fd, nan_value_int),
    cmap=binary_cmap_fd,
    aspect=aspect_ratio,
    extent=(lons.min(), lons.max(), lats.min(), lats.max()),
    vmin=0,
    vmax=1)

axes[0].set_title('FD []')
axes[0].set_ylabel("Lat (wgs84)")
axes[0].set_xlabel("Lon (wgs84)")

# Adjust the colorbar labels for cax1
colorbar_cax1 = fig.colorbar(cax1, ax=axes[0], orientation='vertical', fraction=colorbar_fraction, pad=colorbar_pad)
colorbar_cax1.set_ticks([0, 1])
colorbar_cax1.set_ticklabels(['Not Disturbed', 'Disturbed'])

cax2 = axes[1].imshow(
    np.ma.masked_equal(cfm, nan_value_int),
    cmap=binary_cmap_cfm,
    aspect=aspect_ratio,
    extent=(lons.min(), lons.max(), lats.min(), lats.max()),
    vmin=0,
    vmax=1)


axes[1].set_title('CFM []')
axes[1].set_xlabel("Lon (wgs84)")
axes[1].set_xlabel("Lon (wgs84)")

# Adjust the colorbar labels for cax2
colorbar_cax2 = fig.colorbar(cax2, ax=axes[1], orientation='vertical', fraction=colorbar_fraction, pad=colorbar_pad)
colorbar_cax2.set_ticks([0, 1])
colorbar_cax2.set_ticklabels(['No Forest', 'Forest'])

cax3 = axes[2].imshow(
    np.ma.masked_equal(prob, nan_value_float),
    cmap='rainbow',
    aspect=aspect_ratio,
    extent=(lons.min(), lons.max(), lats.min(), lats.max()),
    vmin=0,
    vmax=1)


axes[2].set_title('Prob of Change []')
axes[2].set_xlabel("Lon (wgs84)")
axes[2].set_ylabel("Lat (wgs84)")


fig.colorbar(cax3, ax=axes[2], orientation='vertical', fraction=colorbar_fraction, pad=colorbar_pad)



fig.tight_layout()
plt.show()

#### Section 5.2: L2B FD  Annotation folder

In [ ]:
acquisition_id_image_path = product_fd_l2b.annotation_acquisition_file   
heatmap_path = product_fd_l2b.annotation_heatmap_file    

#### Section 5.3: L2B FD  Annotation folder - Acquisition ID Image

In [ ]:
acquisition_id_image_layers = {}

with rasterio.open(acquisition_id_image_path) as acquisition_id_image:
    num_channels = acquisition_id_image.count

    for i in range(0, num_channels):
        acquisition_id_image_layers[str(i + 1)] = acquisition_id_image.read(i+1)    
    
    
    # Get the coordinates for the plot
    transform = acquisition_id_image.transform
    width = acquisition_id_image.width
    height = acquisition_id_image.height
    cols, rows = np.meshgrid(np.arange(width), np.arange(height))
    xs, ys = rasterio.transform.xy(transform, rows, cols)
    
    # Convert to numpy arrays
    lons = np.array(xs)
    lats = np.array(ys)

In [ ]:
"""
problem with nan value
"""

nan_value = 0
aspect_ratio =1
colorbar_fraction = 0.05
colorbar_pad = 0.04



fig, axes = plt.subplots(num_channels, 1, figsize=(10, 5 * num_channels))

for i in range(1, num_channels + 1):
    data = acquisition_id_image_layers[str(i)]
    
    # Calculate the aspect ratio
    aspect_ratio = (lons.max() - lons.min()) / (lats.max() - lats.min())
    
    # Plot the data
    ax = axes[i - 1] if num_channels > 1 else axes
    ax1 = ax.imshow(
        np.ma.masked_equal(data, nan_value), 
        cmap="rainbow",
        aspect=aspect_ratio, 
        extent=(lons.min(), lons.max(), lats.min(), lats.max()),
        vmin=0,
        vmax=np.ma.masked_equal(data, nan_value).max()  # Example of using mean and std for vmin and vmax
    )
    ax.set_title(f'Acquisition ID  {i}')
    ax.set_xlabel("Lon (wgs84)")
    ax.set_ylabel("Lat (wgs84)")
    
    # Add colorbar
    fig.colorbar(ax1, ax=ax, orientation='vertical', fraction=colorbar_fraction, pad=colorbar_pad)

fig.tight_layout()
plt.show()


#### Section 5.4: L2B FD  Annotation folder - HeatMap

In [ ]:
with rasterio.open(heatmap_path) as heatmap_raster:
    n_of_L1c_images_contributing = heatmap_raster.read(1)
    n_of_L1c_images_contributing_to_the_final_decision = heatmap_raster.read(2)
    
    
    # Get the coordinates for the plot
    transform = heatmap_raster.transform
    width = heatmap_raster.width
    height = heatmap_raster.height
    cols, rows = np.meshgrid(np.arange(width), np.arange(height))
    xs, ys = rasterio.transform.xy(transform, rows, cols)
    
    # Convert to numpy arrays
    lons = np.array(xs)
    lats = np.array(ys)

In [ ]:
std_multiplier = 3
aspect_ratio = 1
colorbar_fraction = 0.05
colorbar_pad = 0.04
nan_value = 0

fig, axes = plt.subplots(1,2, figsize=(8,8)) 
ax1 = axes[0].imshow(
    np.ma.masked_equal(n_of_L1c_images_contributing,nan_value), 
    cmap='rainbow',
    aspect=aspect_ratio, 
    extent=(lons.min(), lons.max(), lats.min(), lats.max()),
    vmin =  np.ma.masked_equal(n_of_L1c_images_contributing,nan_value).min(),
    vmax =  np.ma.masked_equal(n_of_L1c_images_contributing,nan_value).max()
)
axes[0].set_title('n_of_L1c_images_contributing')
axes[0].set_xlabel("Lon (wgs84)")
axes[0].set_ylabel("Lat (wgs84)")

ax2 = axes[1].imshow(
    np.ma.masked_equal(n_of_L1c_images_contributing_to_the_final_decision, nan_value), 
    cmap='rainbow',
    aspect=aspect_ratio,
    extent=(lons.min(), lons.max(), lats.min(), lats.max()),
    vmin = np.ma.masked_equal(n_of_L1c_images_contributing_to_the_final_decision, nan_value).min(),
    vmax = np.ma.masked_equal(n_of_L1c_images_contributing_to_the_final_decision, nan_value).max()
)
axes[1].set_title('n_of_L1c_images_contributing \n to_the_final decision')
axes[1].yaxis.set_tick_params(labelleft=False)
axes[1].set_xlabel("Lon (wgs84)")

fig.colorbar(ax1, ax=axes[0], orientation='vertical', fraction=colorbar_fraction, pad=colorbar_pad)
fig.colorbar(ax2, ax=axes[1], orientation='vertical', fraction=colorbar_fraction, pad=colorbar_pad)
fig.tight_layout()
plt.show()

## 🎋 Section 6: L2B AGB

#### Section 6.1: L2B FD  COG 

In [ ]:
path_agb = product_agb_l2b.measurement_file   
path_agb_std = product_agb_l2b.measurement_std_file 

print(path_agb)
print(path_agb_std)

In [ ]:
nan_value=product_agb_l2b.floatNoDataValue

with rasterio.open(path_agb) as agb_raster:
    agb_r = agb_raster.read(1)
    
    # Get the coordinates for the plot
    transform = agb_raster.transform
    width = agb_raster.width
    height = agb_raster.height
    cols, rows = np.meshgrid(np.arange(width), np.arange(height))
    xs, ys = rasterio.transform.xy(transform, rows, cols)
    
    # Convert to numpy arrays
    lons = np.array(xs)
    lats = np.array(ys)



with rasterio.open(path_agb_std) as agb_std_raster:
    agb_std_r = agb_std_raster.read(1)

agb_avg = np.nanmean(np.ma.masked_equal(agb_r,nan_value))
agb_std = np.nanstd(np.ma.masked_equal(agb_r, nan_value))

agb_std_avg = np.nanmean(np.ma.masked_equal(agb_std_r, nan_value))
agb_std_std = np.nanstd(np.ma.masked_equal(agb_std_r, nan_value))

In [ ]:
std_multiplier = 1
aspect_ratio = 1
colorbar_fraction = 0.05
colorbar_pad = 0.04
nan_value=product_agb_l2b.floatNoDataValue

fig, axes = plt.subplots(2,1, figsize=(8,8)) 
ax1 = axes[0].imshow(
    np.ma.masked_equal(agb_r,nan_value), 
    cmap='YlGn',
    aspect=aspect_ratio, 
    extent=(lons.min(), lons.max(), lats.min(), lats.max()),
    vmin =  agb_avg - std_multiplier * agb_std,
    vmax =  agb_avg + std_multiplier * agb_std
)
axes[0].set_title('agb [Mg/ha]')
axes[0].set_xlabel("Lon (wgs84)")
axes[0].set_ylabel("Lat (wgs84)")

ax2 = axes[1].imshow(
    np.ma.masked_equal(agb_std_r, nan_value), 
    cmap='rainbow',
    aspect=aspect_ratio,
    extent=(lons.min(), lons.max(), lats.min(), lats.max()),
   vmin =  agb_std_avg - std_multiplier * agb_std_std,
   vmax =  agb_std_avg + std_multiplier * agb_std_std
)
axes[1].set_title('agb std [Mg/ha]')
axes[1].yaxis.set_tick_params(labelleft=False)
axes[1].set_xlabel("Lon (wgs84)")

fig.colorbar(ax1, ax=axes[0], orientation='vertical', fraction=colorbar_fraction, pad=colorbar_pad)
fig.colorbar(ax2, ax=axes[1], orientation='vertical', fraction=colorbar_fraction, pad=colorbar_pad)
fig.tight_layout()
plt.show()


#### Section 6.2: L2B AGB  Annotation folder

In [ ]:
acquisition_id_image_path = product_agb_l2b.annotation_acquisition_file 
fnf_path = product_agb_l2b.annotation_bps_fnf_file
heatmap_path = product_agb_l2b.annotation_heatmap_file

#### Section 6.3: L2B AGB  Annotation folder - FNF

In [ ]:
with rasterio.open(fnf_path) as fnf_raster:
    fnf_array = fnf_raster.read(1)
    
    # Get the coordinates for the plot
    transform = fnf_raster.transform
    width = fnf_raster.width
    height = fnf_raster.height
    cols, rows = np.meshgrid(np.arange(width), np.arange(height))
    xs, ys = rasterio.transform.xy(transform, rows, cols)
    
    # Convert to numpy arrays
    lons = np.array(xs)
    lats = np.array(ys)

In [ ]:
import matplotlib.colors as mcolors
import matplotlib.patches as mpatches
nan_value = 255
aspect_ratio =1


binary_cmap = mcolors.ListedColormap(['brown', 'green'])
white_patch = mpatches.Patch(color='brown', label='No Forest')
black_patch = mpatches.Patch(color='green', label='Forest')

fig, ax = plt.subplots(1, 1)
ax1 = ax.imshow(
    np.ma.masked_equal(fnf_array, nan_value), 
    cmap=binary_cmap,
    aspect=aspect_ratio, 
    extent=(lons.min(), lons.max(), lats.min(), lats.max()),
    vmin=0,
    vmax=1
)
ax.set_title('FNF')
ax.set_xlabel("Lon (wgs84)")
ax.set_ylabel("Lat (wgs84)")

fig.legend(handles=[white_patch, black_patch], loc='right', bbox_to_anchor=(1, 0.5))
fig.tight_layout()
plt.show()


#### Section 6.4: L2B AGB  Annotation folder - HeatMap

In [ ]:
nan_value = 0 ### strange nan_value
with rasterio.open(heatmap_path) as heatmap_raster:
    denominator = heatmap_raster.read(1)
    n_reference_data = heatmap_raster.read(2)
    n_additional_reference_data = heatmap_raster.read(3)
    
    
    # Get the coordinates for the plot
    transform = heatmap_raster.transform
    width = heatmap_raster.width
    height = heatmap_raster.height
    cols, rows = np.meshgrid(np.arange(width), np.arange(height))
    xs, ys = rasterio.transform.xy(transform, rows, cols)
    
    # Convert to numpy arrays
    lons = np.array(xs)
    lats = np.array(ys)

denominator_avg = np.nanmean(np.ma.masked_equal(denominator,nan_value))
denominator_std = np.nanstd(np.ma.masked_equal(denominator, nan_value))

n_reference_data_min = np.nanmin(np.ma.masked_equal(n_reference_data,nan_value))
n_reference_data_max = np.nanmax(np.ma.masked_equal(n_reference_data, nan_value))

n_additional_reference_data_min = np.nanmin(np.ma.masked_equal(n_additional_reference_data,nan_value))
n_additional_reference_data_max = np.nanmax(np.ma.masked_equal(n_additional_reference_data, nan_value))


In [ ]:
n_reference_data_max

In [ ]:
std_multiplier = 1
aspect_ratio = 1
colorbar_fraction = 0.05
colorbar_pad = 0.04
nan_value = 0 

fig, axes = plt.subplots(3,1, figsize=(8,10)) 
ax1 = axes[0].imshow(
    np.ma.masked_equal(denominator,nan_value), 
    cmap='rainbow',
    aspect=aspect_ratio, 
    extent=(lons.min(), lons.max(), lats.min(), lats.max()),
    vmin =  denominator_avg - std_multiplier * denominator_std,
    vmax =  denominator_avg + std_multiplier * denominator_std
)
axes[0].set_title('denominator')
axes[0].set_xlabel("Lon (wgs84)")
axes[0].set_ylabel("Lat (wgs84)")

ax2 = axes[1].imshow(
    np.ma.masked_equal(n_reference_data, nan_value), 
    cmap='rainbow',
    aspect=aspect_ratio,
    extent=(lons.min(), lons.max(), lats.min(), lats.max()),
    vmin = n_reference_data_min,
    vmax = n_reference_data_max
)
axes[1].set_title('n_reference_data')
axes[1].set_xlabel("Lon (wgs84)")
axes[1].set_ylabel("Lat (wgs84)")

ax3 = axes[2].imshow(
    np.ma.masked_equal(n_additional_reference_data, nan_value), 
    cmap='rainbow',
    aspect=aspect_ratio,
    extent=(lons.min(), lons.max(), lats.min(), lats.max()),
    vmin = n_additional_reference_data_min,
    vmax = n_additional_reference_data_max
)
axes[2].set_title('n_additional_reference_data')
axes[2].set_xlabel("Lon (wgs84)")
axes[2].set_ylabel("Lat (wgs84)")

fig.colorbar(ax1, ax=axes[0], orientation='vertical', fraction=colorbar_fraction, pad=colorbar_pad)
fig.colorbar(ax2, ax=axes[1], orientation='vertical', fraction=colorbar_fraction, pad=colorbar_pad)
fig.colorbar(ax3, ax=axes[2], orientation='vertical', fraction=colorbar_fraction, pad=colorbar_pad)
fig.tight_layout()
plt.show()


#### Section 6.5: L2B AGB  Annotation folder - Acquisition ID Image

In [ ]:
acquisition_id_image_layers = {}

with rasterio.open(acquisition_id_image_path) as acquisition_id_image:
    num_channels = acquisition_id_image.count

    for i in range(0, num_channels):
        acquisition_id_image_layers[str(i + 1)] = acquisition_id_image.read(i+1)    
    
    
    # Get the coordinates for the plot
    transform = acquisition_id_image.transform
    width = acquisition_id_image.width
    height = acquisition_id_image.height
    cols, rows = np.meshgrid(np.arange(width), np.arange(height))
    xs, ys = rasterio.transform.xy(transform, rows, cols)
    
    # Convert to numpy arrays
    lons = np.array(xs)
    lats = np.array(ys)

In [ ]:

nan_value = 0
aspect_ratio =1
colorbar_fraction = 0.05
colorbar_pad = 0.04



fig, axes = plt.subplots(num_channels, 1, figsize=(10, 5 * num_channels))

for i in range(1, num_channels + 1):
    data = acquisition_id_image_layers[str(i)]
    
    # Calculate the aspect ratio
    aspect_ratio = (lons.max() - lons.min()) / (lats.max() - lats.min())
    
    # Plot the data
    ax = axes[i - 1] if num_channels > 1 else axes
    ax1 = ax.imshow(
        np.ma.masked_equal(data, nan_value), 
        cmap="rainbow",
        aspect=aspect_ratio, 
        extent=(lons.min(), lons.max(), lats.min(), lats.max()),
        vmin=0,
        vmax=np.ma.masked_equal(data, nan_value).max()  # Example of using mean and std for vmin and vmax
    )
    ax.set_title(f'Acquisition ID  {i}')
    ax.set_xlabel("Lon (wgs84)")
    ax.set_ylabel("Lat (wgs84)")
    
    # Add colorbar
    fig.colorbar(ax1, ax=ax, orientation='vertical', fraction=colorbar_fraction, pad=colorbar_pad)

fig.tight_layout()
plt.show()
